In [ ]:
import numpy as np
import os
from scripts.gen_xrd import create_xrd_tensor
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import Structure
from pymatgen.analysis.diffraction.xrd import XRDCalculator, WAVELENGTHS
import torch
from types import SimpleNamespace
import matplotlib.pyplot as plt
from pymatgen.io.cif import CifParser

In [ ]:
def create_sinc_filter(wave_source, min_2_theta, max_2_theta, xrd_vector_dim, nanomaterial_size):
    wavelength = WAVELENGTHS[wave_source]

    min_theta = min_2_theta / 2
    max_theta = max_2_theta / 2
    Q_min = 4 * np.pi * np.sin(np.radians(min_theta)) / wavelength
    Q_max = 4 * np.pi * np.sin(np.radians(max_theta)) / wavelength

    # phase shift for sinc filter = half of the signed Q range
    phase_shift = (Q_max - Q_min) / 2

    # compute Qs
    Qs = np.linspace(Q_min, Q_max, xrd_vector_dim)
    Qs_shifted = Qs - phase_shift   
    the_filter = nanomaterial_size * np.sinc((np.pi * nanomaterial_size * Qs_shifted)/np.pi)

    return Qs, the_filter

def apply_filter(x, the_filter):
    return np.convolve(x, the_filter, mode='same')

In [ ]:
def create_xrd_args(wave_source, xrd_vector_dim, min_2_theta, max_2_theta):
    alt_args = SimpleNamespace()
    alt_args.wave_source = wave_source
    alt_args.xrd_vector_dim = xrd_vector_dim
    alt_args.max_theta = max_2_theta # misnomer
    alt_args.min_theta = min_2_theta # misnomer

    return alt_args

def generate_xrd(wave_source, curr_structure, symprec, xrd_vector_dim, min_2_theta, max_2_theta):
    curr_wavelength = WAVELENGTHS[wave_source]
    # Create the XRD calculator
    xrd_calc = XRDCalculator(wavelength=curr_wavelength)
    try:
        sga = SpacegroupAnalyzer(curr_structure, symprec=symprec)
        conventional_structure = sga.get_conventional_standard_structure()
    except:
        print(f"Failed to get conventional standard structure for material")
        conventional_structure = curr_structure
    # Calculate the XRD pattern
    try:
        pattern = xrd_calc.get_pattern(conventional_structure)
        # Create the XRD tensor
        xrd_tensor = create_xrd_tensor(create_xrd_args(wave_source=wave_source, 
                                                       xrd_vector_dim=xrd_vector_dim,
                                                       min_2_theta=min_2_theta,
                                                       max_2_theta=max_2_theta), pattern)
    except: 
        print(f"Failed to get XRD pattern for material")
        xrd_tensor = torch.zeros(xrd_vector_dim)
    return xrd_tensor

def read_structure(filepath):
    parser = CifParser(filepath)
    structure = parser.get_structures()[0]
    return structure

def convert_and_save_q_to_2theta(q_xrd, Qs, output_filepath_without_ext,
                        wavelength, plot=False):
    assert len(q_xrd.shape) == 1
    _2thetas = np.degrees(2 * np.arcsin(Qs * wavelength / (4 * np.pi)))
    intensities = q_xrd.numpy()

    with open(f"{output_filepath_without_ext}.xy", 'w') as fout:
        for i in range(len(intensities)):
            fout.write(f'{_2thetas[i]} {intensities[i]}\n')

    plt.plot(_2thetas, intensities)
    plt.savefig(f'{output_filepath_without_ext}_2theta.png')
    plt.close()

    plt.plot(Qs, intensities)
    plt.savefig(f'{output_filepath_without_ext}_q.png')
    plt.close()

    return


In [ ]:
def generate_and_save_xrd_for_cif_file(cif_filepath, output_filepath_without_ext,
                                       wave_source, xrd_vector_dim, min_2_theta, max_2_theta,
                                       nanomaterial_size, plot_xrd):
    curr_structure = read_structure(cif_filepath)
    ideal_xrd_tensor = generate_xrd(wave_source=wave_source, 
                                    curr_structure=curr_structure, 
                                    symprec=0.01, 
                                    xrd_vector_dim=xrd_vector_dim, 
                                    min_2_theta=min_2_theta, 
                                    max_2_theta=max_2_theta)
    Qs, sinc_filter = create_sinc_filter(wave_source=wave_source, 
                                         min_2_theta=min_2_theta, 
                                         max_2_theta=max_2_theta, 
                                         xrd_vector_dim=xrd_vector_dim, 
                                         nanomaterial_size=nanomaterial_size)
    sinc_noised_xrd_array = apply_filter(ideal_xrd_tensor.numpy(), sinc_filter)
    sinc_noised_xrd_tensor = torch.tensor(sinc_noised_xrd_array)

    torch.save(sinc_noised_xrd_tensor, output_filepath_without_ext + '.pt')
    convert_and_save_q_to_2theta(q_xrd=sinc_noised_xrd_array, Qs=Qs, 
                                 output_filepath_without_ext=output_filepath_without_ext,
                                 wavelength=WAVELENGTHS[wave_source],
                                 plot=plot_xrd)
    
    return

In [ ]:
def save_sinc_xrds_for_pregenerated_results(input_dirpath, sinc_size):
    materials = [x for x in os.listdir(input_dirpath) if 'material' in x]
    for curr_material in materials:
        material_num = curr_material.split('_')[0]
        assert 'material' in material_num
        for pred_idx in range(5):
            curr_pred_cif_path = os.path.join(input_dirpath, curr_material, 
                'pred', f'candidate{pred_idx}', 'cif', f'noSpacegroup_{material_num}_candidate{pred_idx}.cif')
            assert os.path.exists(curr_pred_cif_path)
            desired_pred_cif_filepath_without_ext = os.path.join(input_dirpath, curr_material, 
                'pred', f'candidate{pred_idx}', 'xrd_opt_gen', f'sinc{sinc_size}_{curr_material}')
            generate_and_save_xrd_for_cif_file(
                cif_filepath=curr_pred_cif_path, 
                output_filepath_without_ext=desired_pred_cif_filepath_without_ext,
                wave_source='CuKa', 
                xrd_vector_dim=8092, 
                min_2_theta=0, 
                max_2_theta=180,
                nanomaterial_size=sinc_size, 
                plot_xrd=True)
    return

In [ ]:
def save_sinc_xrds_for_curated_results(input_dirpath):
    for root, dirs, files in os.walk(input_dirpath):
        for file in files:
            filepath = os.path.join(root, file)
            assert '.cif' in filepath
            if 'gt' in filepath:
                for sinc_size in [10, 100]:
                    generate_and_save_xrd_for_cif_file(
                        cif_filepath=filepath, 
                        output_filepath_without_ext=os.path.join(root, f'sinc{sinc_size}'),
                        wave_source='CuKa', 
                        xrd_vector_dim=8092, 
                        min_2_theta=0, 
                        max_2_theta=180,
                        nanomaterial_size=sinc_size, 
                        plot_xrd=True
                    )
    return